In [2]:
### computational python libraries 
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import np_utils

from tensorflow.contrib import rnn
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

Using TensorFlow backend.


In [4]:
##read training datasets
buffer=pd.DataFrame()
for i in range(1,2):
    
    subject_01 = pd.read_csv('Datasets_max/P_{0}_win_1000.csv'.format(i))
    
    buffer=pd.concat([buffer,subject_01],axis=0)
### here we read validiation set of subject_20
    subject_8 = pd.read_csv('Datasets_max/P_5_win_1000.csv')



In [11]:
len(subject_01)
print(subject_01.shape)
print(buffer.shape)

(58284, 1001)
(58284, 1001)


In [13]:
#def getdata(X):
    ### drop the label columm to assign data for testingset(train_x) set    
    train_x = buffer.drop('Label',axis=1)
    print(train_x.shape)
    

(58284, 1000)


In [14]:
    
   
    df=train_x
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    train_x = np.asarray(result)
print(train_x.shape)    
    

(58284, 1000)


In [6]:
    train_x = train_x.reshape(train_x.shape[0],train_x.shape[1],1)

    #### here we convert labelled_row data(eg:high_stress,medium_stress,mild_stress) in to int or float using label encoder function
   
    train_yy = X['Label']
    encoder = LabelEncoder()

    ### encoded for train_y data set 
    encoder.fit(train_yy)
    encoded_train_ylabel = encoder.transform(train_yy)
    train_y = np_utils.to_categorical(encoded_train_ylabel)
    return(train_x,train_y)

    

In [9]:
### train data set for model
train_x,train_y = getdata(X=buffer)
### test data set  for model


test_x,test_y = getdata(X=subject_8)

train_x.shape


(58284, 1000, 1)

In [7]:
import keras
def LSTM_RNN_vx(train_xx,train_yy,test_xx,test_yy,units,epoch,batch_size,inputshape,lr):
    #Modeling RNN-LSTM
        model=tf.keras.Sequential()
        model.add(tf.keras.layers.LSTM (units,activation = 'tanh',input_shape =inputshape, return_sequences = True))
        model.add(tf.keras.layers.LSTM (units,activation = 'tanh',input_shape =inputshape, return_sequences = False))
        
        


   
        model.add(tf.keras.layers.Dense(3,activation='sigmoid'))
        #adagrad=keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
       
        model.compile(loss = 'categorical_crossentropy', optimizer = "rmsprop" ,metrics=['acc'])
        hist=model.fit(train_xx,train_yy,epochs=epoch,batch_size=batch_size,shuffle=True,validation_data=(test_xx,test_yy))
        Y_pred=model.predict(test_xx)
        return hist,Y_pred,model

In [8]:
hist,Y_pred,model = LSTM_RNN_vx(train_xx= train_x ,train_yy=train_y,test_xx=test_x,test_yy=test_y,units=10,epoch=10,batch_size=1000,inputshape=(1000,1),lr=0.1)



Instructions for updating:
Colocations handled automatically by placer.
Train on 254420 samples, validate on 59026 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
254420/254420 [==============================] - 644s 3ms/sample - loss: 1.0233 - acc: 0.4804 - val_loss: 1.3102 - val_acc: 0.2955
Epoch 2/10
254420/254420 [==============================] - 725s 3ms/sample - loss: 1.0035 - acc: 0.5081 - val_loss: 1.3803 - val_acc: 0.2826
Epoch 3/10
254420/254420 [==============================] - 687s 3ms/sample - loss: 0.9972 - acc: 0.5121 - val_loss: 1.3228 - val_acc: 0.2851
Epoch 4/10
254420/254420 [==============================] - 828s 3ms/sample - loss: 0.9745 - acc: 0.5385 - val_loss: 1.4422 - val_acc: 0.3016
Epoch 5/10
254420/254420 [==============================] - 881s 3ms/sample - loss: 1.0120 - acc: 0.5210 - val_loss: 1.3966 - val_acc: 0.2988
Epoch 6/10
254420/254420 [==============================] - 750s 3ms/sample - loss: 0.9479 - acc: 0.5599 - val_loss: 1.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\manoj\AppData\Local\conda\conda\envs\tensorflow_env\lib\site-packages\IPython\core\interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-10f4d57adc47>", line 1, in <module>
    hist,Y_pred,model = LSTM_RNN_vx(train_xx= train_x ,train_yy=train_y,test_xx=test_x,test_yy=test_y,units=10,epoch=10,batch_size=1000,inputshape=(1000,1),lr=0.1)
  File "<ipython-input-7-ade0df18b4d9>", line 17, in LSTM_RNN_vx
    Y_pred=model.predict(test_xx)
  File "C:\Users\manoj\AppData\Local\conda\conda\envs\tensorflow_env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1113, in predict
    self, x, batch_size=batch_size, verbose=verbose, steps=steps)
  File "C:\Users\manoj\AppData\Local\conda\conda\envs\tensorflow_env\lib\site-packages\tensorflow\python\keras\engine\training_arrays.py", line 329, in model_iteration
    batch_outs = f(ins_batch)
  File "C:\Users\manoj\A

KeyboardInterrupt: 

In [19]:
model.save('saved/subsampling_model1/intial3.h5')

In [9]:
import matplotlib.pyplot as plt
### loss error plot for train set  
fig = plt.figure()
plt.plot(hist.history['loss'],color = 'g')
plt.legend
plt.title('Train loss')
plt.xlabel('standard output')
plt.ylabel('predicted output')
plt.show()
fig.savefig('plots/model_11_train_loss.png',format='png')
plt.close()

NameError: name 'hist' is not defined

In [ ]:
###  accuray plot for training set
fig = plt.figure()
plt.plot(hist.history['acc'],color = 'red')
plt.legend
plt.title('Train accuary')
plt.xlabel('standard output')
plt.ylabel('predicted output')
plt.show()
fig.savefig('plots/sub_samplingmodel_1_train_acc.png',format='png')
plt.close()

In [ ]:
### loss error for test set or (vadiation set)
fig = plt.figure()
plt.plot(hist.history['val_loss'],color = 'g')
plt.legend
plt.title('valdiation loss')
plt.xlabel('standard output')
plt.ylabel('predicted output')
plt.show()
fig.savefig('plots/sub_samplingmodel_1_val_loss.png',format='png')
plt.close()

In [ ]:
### accuray plot for vadiation set
fig = plt.figure()
plt.plot(hist.history['val_acc'],color = 'red')
plt.legend
plt.title('valdiation accuary')
plt.xlabel('standard output')
plt.ylabel('predicted output')
plt.show()
fig.savefig('plots/sub_samplingmodel_1_val_acc.png',format='png')
plt.close()